In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import pickle
%load_ext zipline

# Setup

```
#
# ~/zipline/extension.py:
#

import pandas as pd

from zipline.data.bundles import register
from zipline.data.bundles.csvdir import csvdir_equities


start_session = pd.to_datetime('2019-09-09 00:00:00+00:00', utc=True)
end_session = pd.to_datetime('2021-05-22 00:00:00+00:00', utc=True)


register(
    'futures',
    csvdir_equities(
        ['minute'],             # daily has some problems
        '/home/XXX/data/futures.csv',
    ),
    calendar_name='24/7',
    minutes_per_day=1440,
    start_session=start_session,
    end_session=end_session,
)
```

# Limit order bug

In [2]:
# Inspect the data, fed to zipline.
uni = pd.read_csv(Path('~/data/futures.csv/minute/UNIUSDT.csv').expanduser())
mask = np.logical_and(
    '2021-01-01'<= uni.date,
                   uni.date < '2021-01-04',
)
uni[mask].describe()

,open,high,low,close,volume,divident,split
count,4320.000000,4320.000000,4320.000000,4320.000000,4320.000000,4320.0,4320.0
mean,4.917989,4.927511,4.908288,4.918016,23407.049769,0.0,1.0
std,0.210354,0.212312,0.208612,0.210472,25554.308038,0.0,0.0
min,4.503400,4.510600,4.486300,4.504200,664.000000,0.0,1.0
25%,4.758775,4.766275,4.749575,4.758475,8755.250000,0.0,1.0
50%,4.864000,4.871150,4.854600,4.863850,15252.000000,0.0,1.0
75%,5.094825,5.104575,5.080675,5.093800,28068.750000,0.0,1.0
max,5.653800,5.660000,5.615000,5.651400,378123.000000,0.0,1.0


In [3]:
%%capture
%%zipline -b futures -s 2021-01-01 -e 2021-01-03 --no-benchmark --data-frequency minute --trading-calendar 24/7 --capital-base 1000000 --output output.pk

from zipline import api


def initialize(context):
    pass


def handle_data(context, data):
    asset = api.symbol('UNIUSDT')
    # What I would expect from this line is to buy
    # just one share (coin in this case) at some point
    # and never again after that.
    api.order_target(asset, 1, limit_price=4.75)

In [4]:
with open('output.pk', 'rb') as f:
    xs = pickle.load(f)
xs['transactions'].map(len)

2021-01-01 23:59:00+00:00    1455
2021-01-02 23:59:00+00:00    1500
2021-01-03 23:59:00+00:00     530
Name: transactions, dtype: int64

In [5]:
xs['orders'].map(len)

2021-01-01 23:59:00+00:00    1440
2021-01-02 23:59:00+00:00    1502
2021-01-03 23:59:00+00:00    1462
Name: orders, dtype: int64

In [6]:
xs[['short_value', 'long_value']]

,short_value,long_value
2021-01-01 23:59:00+00:00,-1.926532e+06,0.0
2021-01-02 23:59:00+00:00,-1.079972e+07,0.0
2021-01-03 23:59:00+00:00,-1.119502e+07,0.0


# ClosePolicy bug

```
I fix would be to close all orders at MINUTE_END,
but it turns out that should_cancel is only called
for SESSION_END and also self.clock never fires
MINUTE_END (tradesimulation.py:252).
```

In [7]:
%%capture
%%zipline -b futures -s 2021-01-01 -e 2021-01-03 --no-benchmark --data-frequency minute --trading-calendar 24/7 --capital-base 1_000_000 --output output.pk

from zipline import api
from zipline.finance.cancel_policy import CancelPolicy
from zipline.gens.sim_engine import MINUTE_END


class EOMCancel(CancelPolicy):
    '''End of minute cancel policy.'''
    def should_cancel(self, event):
        # print(f'EOMCancel.should_cancel: event={event} return={event==MINUTE_END}')
        return event == MINUTE_END


def initialize(context):
    context.i = 0
    api.set_cancel_policy(EOMCancel())


def handle_data(context, data):
    # print('handle_data:', context.get_datetime(), context.i)
    context.i += 1
    asset = api.symbol('UNIUSDT')
    api.order_target(asset, 1, limit_price=4.75)

### OUTPUT
```
handle_data: 2021-01-01 00:00:00+00:00 0
...
handle_data: 2021-01-01 23:59:00+00:00 1439
EOMCancel.should_cancel: event=2 return=False
handle_data: 2021-01-02 00:00:00+00:00 1440
...
handle_data: 2021-01-02 23:59:00+00:00 2879
EOMCancel.should_cancel: event=2 return=False
handle_data: 2021-01-03 00:00:00+00:00 2880
...
handle_data: 2021-01-03 23:59:00+00:00 4319
EOMCancel.should_cancel: event=2 return=False
```

In [8]:
with open('output.pk', 'rb') as f:
    xs = pickle.load(f)
xs['transactions'].map(len)

2021-01-01 23:59:00+00:00    1455
2021-01-02 23:59:00+00:00    1500
2021-01-03 23:59:00+00:00     530
Name: transactions, dtype: int64

In [9]:
xs['orders'].map(len)

2021-01-01 23:59:00+00:00    1440
2021-01-02 23:59:00+00:00    1502
2021-01-03 23:59:00+00:00    1462
Name: orders, dtype: int64

In [10]:
xs[['short_value', 'long_value']]

,short_value,long_value
2021-01-01 23:59:00+00:00,-1.926532e+06,0.0
2021-01-02 23:59:00+00:00,-1.079972e+07,0.0
2021-01-03 23:59:00+00:00,-1.119502e+07,0.0


# Userspace fix

In [11]:
%%capture
%%zipline -b futures -s 2021-01-01 -e 2021-01-03 --no-benchmark --data-frequency minute --trading-calendar 24/7 --capital-base 1000000 --output output.pk

import numpy as np
from zipline import api


def initialize(context):
    pass


def limit_order(asset, amount, limit):
    exists = False
    for order in api.get_open_orders().get(asset, []):
        if np.allclose(order['amount'], amount) and np.allclose(order['limit'], limit):
            exists = True
    if not exists:
        api.order_target(asset, amount, limit)
        
        
def handle_data(context, data):
    asset = api.symbol('UNIUSDT')
    limit_order(asset, 1, 4.75)

In [12]:
with open('output.pk', 'rb') as f:
    xs = pickle.load(f)
xs['transactions'].map(len)

2021-01-01 23:59:00+00:00    1
2021-01-02 23:59:00+00:00    0
2021-01-03 23:59:00+00:00    0
Name: transactions, dtype: int64

In [13]:
xs['orders'].map(len)

2021-01-01 23:59:00+00:00    1
2021-01-02 23:59:00+00:00    0
2021-01-03 23:59:00+00:00    0
Name: orders, dtype: int64

In [14]:
xs[['short_value', 'long_value']]

,short_value,long_value
2021-01-01 23:59:00+00:00,0.0,4.745
2021-01-02 23:59:00+00:00,0.0,4.863
2021-01-03 23:59:00+00:00,0.0,5.460
